In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
sns.set_theme(style="darkgrid")

DIR_DATASETS = r"./data/analytics"

dfFilmes = pd.read_csv(f"{DIR_DATASETS}/titles.csv")
dfElenco = pd.read_csv(f"{DIR_DATASETS}/casting_title.csv")
dfPaises = pd.read_csv(f"{DIR_DATASETS}/countries_title.csv")
dfCategorias = pd.read_csv(f"{DIR_DATASETS}/categories_title.csv")

In [3]:

dfFilmes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18475 entries, 0 to 18474
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       18475 non-null  object
 1   type          18475 non-null  object
 2   title         18475 non-null  object
 3   date_added    8864 non-null   object
 4   release_year  18475 non-null  int64 
 5   rating        18134 non-null  object
 6   duration      18472 non-null  object
 7   description   18475 non-null  object
 8   title_id      18475 non-null  object
 9   platform      18475 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.4+ MB


In [5]:
dfElenco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108493 entries, 0 to 108492
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   title_id  108493 non-null  object
 1   artist    108490 non-null  object
dtypes: object(2)
memory usage: 1.7+ MB


In [7]:
top10Atores = dfElenco.groupby(["artist"])["artist"].count().nlargest(10)
top10Atores

artist
Anupam Kher         59
Maggie Binkley      56
Amitabh Bachchan    47
Shah Rukh Khan      46
Nassar              43
Akshay Kumar        41
Danny Trejo         39
Naseeruddin Shah    39
Paresh Rawal        39
1                   35
Name: artist, dtype: int64

In [10]:
producoesPorPaisPlataforma = pd.pivot_table(
    dfPaises,
    values=["title_id"],
    index=["country"],
    columns=["platform"],aggfunc="count",fill_value=0, margins = True, margins_name='total')

producoesPorPaisPlataforma.nlargest(5,[('title_id','total')])

title_id               
platform       amazon_prime netflix  total
country                                   
total                   824   10012  10836
United States           334    3690   4024
India                   246    1046   1292
United Kingdom           67     806    873
Canada                   35     445    480

In [11]:
df = dfPaises.groupby(["country","platform"]).count()
df

title_id
country      platform              
Afghanistan  amazon_prime         1
             netflix              1
Albania      amazon_prime         1
             netflix              1
Algeria      netflix              3
...                             ...
Vatican City netflix              1
Venezuela    netflix              4
Vietnam      netflix              7
West Germany netflix              5
Zimbabwe     netflix              3

[167 rows x 1 columns]

In [ ]:
dfFilmesPorAno = dfFilmes.groupby(['release_year'], as_index=False)['id'].count()
f, ax = plt.subplots(figsize=(10, 5))

ax = sns.lineplot(x='release_year',y='id',data=dfFilmesPorAno,markers=True, dashes=False)
ax.set_title('Filmes por ano de lançamento')
ax.set_xlabel('Ano')
ax.set_ylabel('Quantidade')
ax.plot()

dfFilmesPorAno